In [1]:
import pandas as pd

In [3]:
rename_columns = {
    "porto_alegre": "Porto Alegre", 
    "brasilia": "Brasília", 
    "maraba": "Marabá", 
    "juazeiro_do_norte": "Juazeiro do Norte",
    "recife": "Recife", 
    "belo_horizonte": "Belo Horizonte"
}

df = pd.read_csv("results.csv").rename(columns=rename_columns)

In [4]:
test_columns = ["porto_alegre", "brasilia", "maraba", "juazeiro_do_norte", "recife", "belo_horizonte"]

# Function to get the name of the column with "Teste" for each row
def get_test_column(row):
    for col in test_columns:
        if row[col] == 'Teste':
            return col
    return None  # If no column has "Teste" in the row

# Create a new column with the name of the column containing "Teste"
df["hub"] = df.apply(get_test_column, axis=1)

In [6]:
df

,Unnamed: 0,type,porto_alegre,brasilia,maraba,juazeiro_do_norte,recife,belo_horizonte,auc_score,f1_score,acc,recall,precision,hub
0,0,xgb_split3_2,Teste,Treino,Treino,Validação,Validação,Treino,0.817342,0.222435,0.817342,0.140597,0.532246,porto_alegre
1,1,xgb_split3_2,Teste,Treino,Treino,Treino,Validação,Validação,0.795590,0.308953,0.795590,0.331358,0.289385,porto_alegre
2,2,xgb_split3_2,Teste,Treino,Treino,Validação,Treino,Validação,0.821250,0.378626,0.821250,0.363814,0.394696,porto_alegre
3,3,xgb_split3_2,Teste,Validação,Treino,Treino,Validação,Treino,0.788255,0.000000,0.788255,0.000000,0.000000,porto_alegre
4,4,xgb_split3_2,Teste,Validação,Treino,Validação,Treino,Treino,0.835430,0.325797,0.835430,0.229479,0.561450,porto_alegre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,91,ensemble,Treino,Treino,Teste,Treino,Treino,Treino,0.599624,0.436946,0.309112,0.981086,0.281061,maraba
92,92,ensemble,Treino,Teste,Treino,Treino,Treino,Treino,0.766492,0.299403,0.539019,0.904400,0.179396,brasilia
93,93,ensemble,Treino,Treino,Treino,Teste,Treino,Treino,0.546829,0.346650,0.234917,0.950373,0.211986,juazeiro_do_norte
94,94,ensemble,Treino,Treino,Treino,Treino,Teste,Treino,0.785826,0.426209,0.331811,0.994377,0.271233,recife
